In [122]:
import numpy as np
import pandas as pd
import json
from matplotlib import pyplot as plt
from sklearn import manifold

In [123]:
with open('../files/ls.json') as f:
    data_file = json.load(f)

In [124]:
data_filter = filter(lambda d: d['type'] == 'LoadConst' and d['params'] == 'edi', data_file)
_data = []
for d in data_filter:
    _data.append(d)

In [125]:
data = pd.io.json.read_json(json.dumps(_data))
data

address  address_end  arch dest  \
0      20669        20670    32  edi   
1      59869        59873    32  edi   
2      16926        16930    32  edi   
3      16949        16953    32  edi   
4      48721        48725    32  edi   
5      16939        16941    32  edi   
6      16358        16360    32  edi   
7      47487        47491    32  edi   
8      59868        59873    32  edi   
9      59913        59918    32  edi   
10     16925        16930    32  edi   
11     16948        16953    32  edi   
12     29609        29616    32  edi   
13     48709        48715    32  edi   
14     48711        48715    32  edi   
15     48720        48725    32  edi   
16     30302        30309    32  edi   
17     30323        30332    32  edi   
18     43570        43574    32  edi   
19     16938        16941    32  edi   
20     36131        36136    32  edi   
21     48708        48715    32  edi   
22     44654        44658    32  edi   
23     44865        44869    32  edi   
24     16357        16360    32  edi   
25     18759        18765    32  edi   
26     41124        41132    32  edi   
27     29608        29616    32  edi   
28     47486        47491    32  edi   
29     47519        47528    32  edi   
..       ...          ...   ...  ...   
156    16945        16953    32  edi   
157    28329        28335    32  edi   
158    68021        68032    32  edi   
159    29579        29587    32  edi   
160    29581        29587    32  edi   
161    86952        86963    32  edi   
162    21651        21662    32  edi   
163    21655        21662    32  edi   
164    24183        24190    32  edi   
165    52969        52980    32  edi   
166    20214        20222    32  edi   
167    20216        20222    32  edi   
168    83811        83822    32  edi   
169    37379        37386    32  edi   
170    16924        16930    32  edi   
171    26408        26415    32  edi   
172    30300        30309    32  edi   
173    22335        22346    32  edi   
174    68024        68032    32  edi   
175    41122        41132    32  edi   
176    59862        59873    32  edi   
177    59966        59977    32  edi   
178    40430        40440    32  edi   
179    44881        44887    32  edi   
180    28317        28335    32  edi   
181    28323        28335    32  edi   
182    28321        28335    32  edi   
183    29574        29587    32  edi   
184    16921        16930    32  edi   
185    52972        52980    32  edi   

                                                disasm  \
0                                         pop edi; ret   
1                        pop edi; cmovne eax, edx; ret   
2                               setne al; pop edi; ret   
3                               pop edi; setne al; ret   
4                                pop edi; sete al; ret   
5                                pop esi; pop edi; ret   
6                                pop edi; pop ebp; ret   
7                  xor eax, eax; pop esi; pop edi; ret   
8               pop esi; pop edi; cmovne eax, edx; ret   
9               cmovne eax, edx; pop esi; pop edi; ret   
10                     pop esi; setne al; pop edi; ret   
11                     pop esi; pop edi; setne al; ret   
12          mov eax, 0xffffffff; pop esi; pop edi; ret   
13    or al, 0x74; add al, 0x5b; pop esi; pop edi; ret   
14                 add al, 0x5b; pop esi; pop edi; ret   
15                      pop esi; pop edi; sete al; ret   
16                   mov eax, 1; pop esi; pop edi; ret   
17   or al, 0x5b; mov eax, 0xffffffff; pop esi; pop...   
18                clc ; pop esi; pop edi; pop ebp; ret   
19                      pop ebx; pop esi; pop edi; ret   
20        cmp al, 0x5b; pop esi; pop edi; pop ebp; ret   
21   pop edi; or al, 0x74; add al, 0x5b; pop esi; p...   
22             dec ebx; pop ebx; pop esi; pop edi; ret   
23                clc ; pop ebx; pop esi; pop edi; ret   
24                      pop esi; pop edi; pop ebp; ret   
25       add esp

In [126]:
data['mem'][13][0]

[]

In [127]:
data['modified_regs'][13]

['eax', 'edi', 'esi']

In [128]:
dissM=np.zeros((len(data),len(data))) #creates a zeros dissM
for i in range(len(data)):
    for j in range (len(data)):
        a1 = set(data['modified_regs'][i])
        b1 = set(data['modified_regs'][j])
        a2 = set(data['mem'][i][0])
        b2 = set(data['mem'][j][0])
        dissM[i][j]= len((a1 | b1) - (a1 & b1))# + len((a2 | b2) - (a2 & b2))

In [135]:
mds = manifold.MDS(n_components=2, max_iter=3000, eps=1e-9,
                   dissimilarity="precomputed")
pos = mds.fit(dissM).embedding_

s = 50
plt.scatter(pos[:, 0], pos[:, 1], color='red',s=s, lw=0, label='Gadgets')

In [136]:
for label, x, y in zip(data['modified_regs'], pos[:, 0], pos[:, 1]):
    plt.annotate(
        label,
        xy = (x, y), xytext = (-20, 20),
        textcoords = 'offset points', ha = 'right', va = 'bottom',
        bbox = dict(boxstyle = 'round,pad=0.3', fc = 'yellow', alpha = 0.5),
        arrowprops = dict(arrowstyle = '->', connectionstyle = 'arc3,rad=0'))
plt.legend()

In [137]:
plt.show()